## Tool to read the labels of the 42 mice annotated by the expert

The labels for each mice are stored in txt files. The goal of this notebook is to read the labels and store them in a dataframe.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn.metrics as metrics

import sys
sys.path.append('../Library')
import breedManip as breedManip
import dataProcessing as dataProcessing
import breeds as breeds
import splitData as splitData

import pickle
import os

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)
2023-07-23 17:02:42.659871: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-23 17:02:42.735463: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-23 17:02:43.900833: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [8]:
folder = "/home/magali.egger/shared-projects/mice_UNIL/BXD envoie/TXT_files/"
mice_annotated = ['BL601', 'BL602', 'BL606', 'BL609', '1D203', '2D203', '1D204', '1D206', '00505', '04501', '04502', '04504', '04505', '04506', '04508', '04801', '04802', '04803', '04804', '04805', '04806', '05001', '05002', '05003', '05004', '05005', '05006', '07901', '07902', '08405', '08502', '08503', '08901', '08902', '09001', '09003', '09004', '09005', '09006', '09007', '10101', '10102']

In [ ]:
# create a dataframe with three columns (epoch, mouse, label)
df_labels = pd.DataFrame(columns=['epoch', 'mouse', 'label'])

for mouse in mice_annotated:
    file = folder + mouse + ".txt"
    df_mouse = pd.read_csv(file, sep="\t", header=None)
    df_mouse.columns = ['label']
    df_mouse['mouse'] = mouse
    df_mouse['epoch'] = df_mouse.index
    df_labels = df_labels.append(df_mouse) # type: ignore

In [30]:
df_labels['label'].value_counts()

w    1642790
n    1424139
1     300856
r     191709
2      40776
3      11509
5       6185
8       5533
4       3267
9        954
6        622
s        457
a          2
j          1
Name: label, dtype: int64

In [31]:
df_labels.head()

,epoch,mouse,label
0,0,BL601,w
1,1,BL601,w
2,2,BL601,w
3,3,BL601,w
4,4,BL601,w


In [32]:
df_labels.describe()

,epoch,mouse,label
count,3628800,3628800,3628800
unique,86400,42,14
top,0,BL601,w
freq,42,86400,1642790


In [33]:
with open('/home/magali.egger/workspace/TBproject/Travail_Bachelor/Data/df_labels.pkl', 'wb') as f:
    pickle.dump(df_labels, f)

In [4]:
with open('/home/magali.egger/shared-projects/mice_UNIL/Data/df_labels.pkl', 'rb') as f:
    df_labels = pickle.load(f)

Explore the mice and their breeds from the mice annotated by the expert.

In [34]:
mice_data = breedManip.getAllmiceInAllBreeds()

# display the mice that are new from the 42 annotated mice
for mouse in mice_annotated:
    if mouse not in mice_data:
        print(mouse)

05005
05006


In [35]:
# get all the breeds of the mice : breedManip.getBreedOfMouse(mouse)
breeds = []
new_mice = ['05005', '05006']
for mouse in mice_annotated:
    if mouse not in new_mice:
        breeds.append(breedManip.getBreedOfMouse(mouse))

print("Breeds of the 42 annotated mice", set(breeds))

Breeds of the 42 annotated mice {'bxd_050', 'c57bl6', 'bxd_079', 'bxd_084', 'bxd_090', 'bxd_101', 'bxd_085', 'bxd_048', 'bxd_005', 'dba2', 'bxd_089', 'bxd_045'}


#### Creation of a dataframe that merge the features and the labels of the 42 mice annotated by the expert

In [6]:
# get the df_simplify 
with open('/home/magali.egger/shared-projects/mice_UNIL/Data/df_filter.pkl', 'rb') as f:
    df_simplify = pickle.load(f)

# get the df_nofilter
with open('/home/magali.egger/shared-projects/mice_UNIL/Data/df_full_noFilter.pkl', 'rb') as f:
    df_nofilter = pickle.load(f)

# get the df_labels
with open('/home/magali.egger/shared-projects/mice_UNIL/Data/df_labels.pkl', 'rb') as f:
    df_labels = pickle.load(f)

In [9]:
df_simplify = df_simplify[df_simplify['mouse'].isin(mice_annotated)]
df_nofilter = df_nofilter[df_nofilter['mouse'].isin(mice_annotated)]

In [10]:
df_simplify_labels = pd.merge(df_simplify, df_labels, on=['mouse', 'epoch'])
df_nofilter_labels = pd.merge(df_nofilter, df_labels, on=['mouse', 'epoch'])

In [11]:
# test if the merge worked by comparing the values of the labels 
# with the ones of the rawState on the third day
df_simplify_labels_day2 = df_simplify_labels[df_simplify_labels['day'] == 2]
for index, row in df_simplify_labels_day2.iterrows():
    if row['label'] != row['rawState']:
        print(row['mouse'], row['epoch'], row['label'], row['rawState'])

In [12]:
df_nofilter_labels.head()

,Unnamed: 0,rawState,state,EEGv,EMGv,epoch,day,spectral_flatness,spectral_centroid,spectral_entropy,...,EEGv_log,EMGv_log,bias,EEGv^2,EEGv^3,EMGv^2,EMGv^3,mouse,breed,label
0,99,n,n,7.116438e-09,2.315795e-10,99,0,0.054175,9.400168,-5.994889,...,-18.760859,-22.186098,1,5.064369e-17,3.604026e-25,5.362906e-20,1.241939e-29,09003,bxd_090,n
1,100,n,n,6.789003e-09,1.883032e-10,100,0,0.075867,9.576805,-5.910677,...,-18.807962,-22.392967,1,4.609056e-17,3.129089e-25,3.545811e-20,6.676878e-30,09003,bxd_090,n
2,101,n,n,5.345885e-09,1.965400e-10,101,0,0.082830,10.181243,-5.931538,...,-19.046939,-22.350155,1,2.857849e-17,1.527773e-25,3.862797e-20,7.591942e-30,09003,bxd_090,n
3,102,n,n,8.861552e-09,1.982735e-10,102,0,0.052586,7.561301,-5.506340,...,-18.541544,-22.341374,1,7.852710e-17,6.958720e-25,3.931238e-20,7.794603e-30,09003,bxd_090,n
4,103,n,n,6.617297e-09,2.121310e-10,103,0,0.068133,8.691670,-5.728127,...,-18.833579,-22.273817,1,4.378862e-17,2.897624e-25,4.499956e-20,9.545802e-30,09003,bxd_090,n


In [13]:
# test if the merge worked by comparing the values of the labels
# with the ones of the rawState on the third day
df_nofilter_labels_day2 = df_nofilter_labels[df_nofilter_labels['day'] == 2]
for index, row in df_nofilter_labels_day2.iterrows():
    if row['label'] != row['rawState']:
        print(row['mouse'], row['epoch'], row['label'], row['rawState'])

In [15]:
with open('/home/magali.egger/shared-projects/mice_UNIL/Data/df_simplify_labels.pkl', 'wb') as f:
    pickle.dump(df_simplify_labels, f)

with open('/home/magali.egger/shared-projects/mice_UNIL/Data/df_nofilter_labels.pkl', 'wb') as f:
    pickle.dump(df_nofilter_labels, f)

In [5]:
with open('/home/magali.egger/workspace/TBproject/Travail_Bachelor/Data/df_simplify_labels.pkl', 'rb') as f:
    df_simplify_labels = pickle.load(f)

In [8]:
# compare df_simplify with df_simplify_labels
df_simplify = df_simplify[df_simplify['mouse'].isin(mice_annotated)]


In [ ]:
# compare df_simplify with df_simplify_labels


(df_simplify['EEGv']).equals(df_simplify_labels['EEGv'])

for index, row in df_simplify.iterrows():
    if row['EEGv'] != df_simplify_labels.iloc[index]['EEGv']:
        print(row['mouse'], row['epoch'], row['EEGv'], df_simplify_labels.iloc[index]['EEGv'])